## Event reproducibility, manifold attractors, and functional connectivity analysis of Spiking simulations

Saved data only here, to run the simulation please refere to the companion repository.

- random connectivity: events but not reproducibility
- range connectivity:
    - event reproducibility
    - manifold attractors
    - cores and functional connectivity

In [14]:
from platform import python_version
print(python_version())

%run -i 'imports_functions.py' 

# import collections
# collections.MutableSequence = collections.abc.MutableSequence
# import neo
# import quantities as pq

%matplotlib inline

# import pkg_resources
# for m in pkg_resources.working_set:
#     print(m)

3.10.4


In [17]:
if os.path.exists('simulations/spiketrains_py.npy'):
    all_spiketrains = np.load('simulations/spiketrains_py.npy', allow_pickle=True)
print(all_spiketrains)


[array([7.420000e+01, 9.090000e+01, 1.539000e+02, 1.579900e+03,
        1.608300e+03, 1.644500e+03, 2.762000e+03, 2.774200e+03,
        2.823200e+03, 3.995000e+03, 4.099100e+03, 4.125200e+03,
        5.484800e+03, 5.496500e+03, 5.526600e+03, 5.552000e+03,
        8.615700e+03, 8.647200e+03, 9.528800e+03, 9.543300e+03,
        9.568900e+03, 1.073390e+04, 1.145490e+04, 1.147850e+04,
        1.149920e+04, 1.249920e+04, 1.341000e+04, 1.342560e+04,
        1.446150e+04, 1.472060e+04, 1.473680e+04, 1.628320e+04,
        1.630710e+04, 1.636690e+04, 1.640590e+04, 1.748700e+04,
        1.750730e+04, 1.752120e+04, 1.838330e+04, 1.840890e+04,
        1.929950e+04, 1.931470e+04, 2.124410e+04, 2.134880e+04,
        2.215900e+04, 2.220630e+04, 2.222880e+04, 2.276330e+04,
        2.279030e+04, 2.399590e+04, 2.506960e+04, 2.668340e+04,
        2.669920e+04, 2.671190e+04, 2.672470e+04, 2.741340e+04,
        2.899550e+04, 3.054960e+04, 3.056460e+04, 3.343930e+04,
        3.478230e+04, 3.592880e+04, 3.63